<a href="https://colab.research.google.com/github/aaron23gregory/Preppin-Data-Challenges/blob/main/Preppin_Data_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Week 3 Target DSB

Data Source Bank has some quarterly targets for the value of transactions that are being performed in-person and online. It's our job to compare the transactions to these target figures.

Requirements
Input the data
For the transactions file:
- Filter the transactions to just look at DSB. These will be transactions that contain DSB in the Transaction Code field

- Rename the values in the Online or In-person field, Online of the 1 values and In-Person for the 2 values
- Change the date to be the quarter 
- Sum the transaction values for each quarter and for each Type of Transaction (Online or In-Person)

For the targets file:
- Pivot the quarterly targets so we have a row for each Type of Transaction and each Quarter
- Remove the 'Q' from the quarter field and make the data type numeric 
- Join the two datasets together
- Remove unnecessary fields
- Calculate the Variance to Target for each row
Output the data

In [1]:
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()

Saving PD 2023 Wk 1 Input.csv to PD 2023 Wk 1 Input.csv


In [2]:
df = pd.read_csv(io.BytesIO(uploaded['PD 2023 Wk 1 Input.csv']))
df.head()  

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
0,DTB-716-679-576,1448,100001,2,20/03/2023 00:00:00
1,DS-795-814-303,7839,100001,2,15/11/2023 00:00:00
2,DSB-807-592-406,5520,100005,1,14/07/2023 00:00:00
3,DS-367-545-264,7957,100007,2,18/08/2023 00:00:00
4,DSB-474-374-857,5375,100000,2,26/08/2023 00:00:00


In [3]:
uploaded = files.upload()

Saving Targets.csv to Targets.csv


In [4]:
df_targets = pd.read_csv(io.BytesIO(uploaded['Targets.csv']))
df_targets.head()  

,Online or In-Person,Q1,Q2,Q3,Q4
0,Online,72500,70000,60000,60000
1,In-Person,75000,70000,70000,60000


In [5]:
#Add a bank column so we can select only "DSB" from the transaction codes
df['Bank'] = df['Transaction Code'].apply(lambda x: x.split("-")[0])
df=df[df["Bank"]=="DSB"]
df.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date,Bank
2,DSB-807-592-406,5520,100005,1,14/07/2023 00:00:00,DSB
4,DSB-474-374-857,5375,100000,2,26/08/2023 00:00:00,DSB
5,DSB-448-546-348,4525,100009,1,27/05/2023 00:00:00,DSB
11,DSB-422-218-322,118,100010,1,12/05/2023 00:00:00,DSB
12,DSB-669-227-170,830,100001,1,15/04/2023 00:00:00,DSB


In [6]:
#Replacing the 1's and 2's in "Online or in person" with "Online" or "In-Person"
replacement_mapping = {1: 'Online', 2: 'In-Person'}
df['Online or In-Person'] = df['Online or In-Person'].map(replacement_mapping)
df.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date,Bank
2,DSB-807-592-406,5520,100005,Online,14/07/2023 00:00:00,DSB
4,DSB-474-374-857,5375,100000,In-Person,26/08/2023 00:00:00,DSB
5,DSB-448-546-348,4525,100009,Online,27/05/2023 00:00:00,DSB
11,DSB-422-218-322,118,100010,Online,12/05/2023 00:00:00,DSB
12,DSB-669-227-170,830,100001,Online,15/04/2023 00:00:00,DSB


In [7]:
#Change the transaction date to a day of the week
df['Transaction Date'] = pd.to_datetime(df['Transaction Date']) 
df["Quarter"] = df['Transaction Date'].dt.quarter
df.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date,Bank,Quarter
2,DSB-807-592-406,5520,100005,Online,2023-07-14,DSB,3
4,DSB-474-374-857,5375,100000,In-Person,2023-08-26,DSB,3
5,DSB-448-546-348,4525,100009,Online,2023-05-27,DSB,2
11,DSB-422-218-322,118,100010,Online,2023-12-05,DSB,4
12,DSB-669-227-170,830,100001,Online,2023-04-15,DSB,2


In [22]:
#Sum the transaction values for each quarter and for each Type of Transaction (Online or In-Person)
group_df= df.groupby(['Online or In-Person', 'Quarter'])
print(group_df['Value'].sum())

Online or In-Person  Quarter
In-Person            1          47115
                     2          81627
                     3          84048
                     4          52832
Online               1          55870
                     2          94860
                     3          56617
                     4          57520
Name: Value, dtype: int64


In [24]:
#Pivot the quarterly targets so we have a row for each Type of Transaction and each Quarter
pivoted_targets = df_targets.melt(id_vars='Online or In-Person', var_name='Quarter', value_name='Value')
pivoted_targets.head()

,Online or In-Person,Quarter,Value
0,Online,Q1,72500
1,In-Person,Q1,75000
2,Online,Q2,70000
3,In-Person,Q2,70000
4,Online,Q3,60000


In [26]:
# Remove the 'Q' from the quarter field and make the data type numeric
pivoted_targets['Quarter']=pivoted_targets['Quarter'].apply(lambda x: x[1])
pivoted_targets.head()

,Online or In-Person,Quarter,Value
0,Online,1,72500
1,In-Person,1,75000
2,Online,2,70000
3,In-Person,2,70000
4,Online,3,60000


In [23]:
# Join the two datasets together
pivoted_table['Quarter'] = pivoted_table['Quarter'].apply(pd.to_numeric)
merged_df = pd.merge(group_df, pivoted_table, on=[['Online or In-Person','Quarter']], how='inner')
merged_df

TypeError: ignored